In [7]:
import tensorflow
import gym
import random
import atari_py
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy



from gym import Env
from gym.spaces import Discrete, Box
import random

import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [8]:
def norm_fun( x): 
    
    high    = x[0]
    low     = x[0]
    new_x = []

    for n in range(0, len(x)):
        if ( x[n] > high):
            high = x[n]
        if ( x[n] < low):
            low = x[n]
                
    for n in range(0, len(x)):
        new_x.append((x[n] -low) - (high - low))

    return new_x


In [9]:
nsdq    = pd.read_csv("Nsdq.csv")
spy     = pd.read_csv("Spy.csv")
combi   = pd.merge(nsdq, spy, on = 'Date')
combi["dif_x"] = combi["Open_x"] - combi["Close_x"]
combi["dif_y"] = combi["Open_y"] - combi["Close_y"]

combi = combi.dropna()

tot_labels    = []
nsdq_train   = []
spy_train = []

sum_ = 0 
dif = []

for x in range (1, len(combi)):
            
    if(combi.iloc[x]["Open_x"] > combi.iloc[x]["Close_x"] ):
        tot_labels.append([1])
        sum_ = sum_ +1
        dif.append(combi.iloc[x]["Open_x"] - combi.iloc[x]["Close_x"] )
    else:
        tot_labels.append([0]) 
        dif.append(combi.iloc[x]["Open_x"] - combi.iloc[x]["Close_x"] )
        sum_ = sum_ -1
    temp_list_x =norm_fun( [combi.iloc[x-1]["Open_x"], combi.iloc[x-1]["Close_x"], 
                                    combi.iloc[x-1]["High_x"], combi.iloc[x-1]["Low_x"], combi.iloc[x]["Open_x"]])
    temp_list_y = norm_fun([combi.iloc[x-1]["Open_y"], combi.iloc[x-1]["Close_y"], 
                                    combi.iloc[x-1]["High_y"], combi.iloc[x-1]["Low_y"]] )

            
    nsdq_train.append( temp_list_x )
    spy_train.append( temp_list_y )

        
nsdq_train = tf.keras.utils.normalize(nsdq_train, axis =1)
spy_train = tf.keras.utils.normalize(spy_train, axis=1)

tot_train = np.concatenate((nsdq_train,spy_train), axis = 1)



tot_labels = np.array(tot_labels)
encoder = LabelEncoder()
encoder.fit(tot_labels)
encoded_Y = encoder.transform(tot_labels)


print(dif)

[-23.75, 2.25, 13.0, -2.25, 48.5, 7.0, 8.5, 46.0, 81.75, -81.5, -8.75, -82.25, -10.25, -32.75, 13.75, 7.75, -12.25, -2.75, 16.75, -37.5, 20.5, 87.75, -97.25, 20.25, 12.5, 74.5, -20.0, 107.0, -32.0, -34.5, -55.5, 79.0, 1.0, -49.25, -22.0, -16.25, -3.25, -2.75, -72.75, 41.25, 60.25, 88.0, 21.75, 35.25, 161.0, 36.5, -8.75, -40.5, 133.0, -76.0, 114.25, -17.75, 8.0, 0.5, -121.25, 60.75, -36.75, 68.25, -3.5, -128.0, -28.25, 94.5, 14.5, 14.0, 124.5, 63.0, 14.25, -12.75, 1.75, -43.0, -71.25, -107.0, 53.25, -4.75, -59.5, 48.0, -52.0, -29.75, 13.0, 24.0, -138.5, 5.25, 6.5, 0.5, 21.0, 30.5, -23.5, 10.25, -65.5, -13.25, 0.25, -31.25, -5.5, -9.5, -21.5, -18.5, 39.25, -2.25, 12.0, -68.5, -21.25, 4.0, -50.75, 26.5, 30.25, -59.0, 63.25, 0.25, 18.5, -43.0, -58.75, 0.25, 10.25, -27.75, 18.75, -7.5, 35.5, 22.5, 4.75, 19.75, -18.75, 50.0, 52.0, -32.0, 37.5, 25.75, 8.0, -29.0, -6.0, -57.25, 42.75, 17.75, 13.25, -59.25, 53.25, -11.25, 20.5, -45.25, 15.25, -93.75, -29.25, -18.75, -16.0, -6.0, 0.5, -11.75, 21

In [10]:
class ShowerEnv(Env):
    



    def get_y(self, time):

        y_out = encoded_Y[time]

        return y_out     

    def __init__(self):

        

        self.time = 0

        
        self.cash = 3000
        
        
        self.action_space = Discrete(2)

        self.state = np.reshape(tot_train[self.time], -1)
        print(self.state)
        self.shape = (10,)


        



    def step(self, action):
        self.time +=1
        self.state = tot_train[self.time]

        y = self.get_y(self.time)
        #print(action, y)
        if( y == action):
            reward = abs(dif[self.time])
            self.cash += reward
        else:
            reward = -abs(dif[self.time-1])
            self.cash += reward

        if self.time >= 1000:
            done = True
        else:
            done = False

        info = {}
        #print (self.cash)
        return self.state, reward, done, info
        

    def render(self):
        pass
    def reset(self):
        self.time = 0
        self.cash = 3000
        self.state = tot_train[self.time]
        return self.state

    

In [11]:
env = ShowerEnv()

[-0.58857833 -0.05885783  0.         -0.79725611 -0.12039102 -0.68283699
 -0.06828287  0.         -0.72737274]


In [12]:
env.action_space.sample()

1

In [13]:
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} score:{}'.format(episode, score))
env.close()

Episode:1 score:2094.129882
Episode:2 score:1151.7099609999996


In [14]:
states= (9,)
actions = env.action_space.n

In [15]:
def build_model(states, actions):
    model = Sequential()



    model.add(Flatten(input_shape = (2,9)))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(actions, activation = 'softmax'))
    return model


In [16]:
model = build_model(states, actions)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 18)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                608       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 66        
Total params: 25,538
Trainable params: 25,538
Non-trainable params: 0
____________________________________________________

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    
    memory = SequentialMemory(limit =100, window_length =2)
    
    dqn = DQNAgent(model = model,
                   memory=memory,
                   policy=policy, 
                   
                   nb_actions = actions,
                   nb_steps_warmup =1000,
                   target_model_update=1e-2
                  )
    return dqn

    
    

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4), metrics=['mae'])
dqn.fit(env, nb_steps = 100000, visualize=False, verbose =1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
 2758/10000 [=======>......................] - ETA: 30s - reward: 3.6370done, took 11.463 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=1, visualize=False)
print(np.mean(scores.history['episode_reward']))